In [ ]:
import pandas as pd
import numpy as np
import os
import pronouncing
import speech_recognition as sr
import Levenshtein
import difflib
from speech_recognition_system import SpeechRecognitionSystem
from gtts import gTTS
from playsound import playsound
from IPython.display import Audio,display

In [ ]:
# We load custom functions from a python file
from extra.utility import find_sentence_with_word

In [ ]:
import sys
sys.path.append('/Users/mac/Spiced/Projects/ISP_International_Speech_Project/extra/utility.py')


### # Reading the text from the input folder

In [ ]:
# Reading the text from the input folder
texts = pd.read_csv('data/sentences_test.csv')
texts = [x for x in texts['text']]
texts

### Audio logic test explanation 

In [ ]:
def play_audio(text):
        # Create a gTTS object
        tts = gTTS(text=text, lang='en', slow=False)

        # Save the generated audio file
        audio_file_path = '/Users/mac/Spiced/Projects/ISP_International_Speech_Project/data/output_audio_test.mp3'
        tts.save(audio_file_path)

        # Play the saved audio file
        playsound(audio_file_path)

text = "hello world I am a robot"
play_audio(text)

### Audio with the main script

In [ ]:
# Reading the text from the input folder
texts = pd.read_csv('data/sentences_a.csv')['Sentences'].tolist()

# Reading the words from the input folder
unique_word_dict = pd.read_csv('data/unique_words_a.csv')['about'].tolist()

# Create an instance of the SpeechRecognitionSystem
speech_recognition_system = SpeechRecognitionSystem(texts, unique_word_dict)

# Create a gTTS object
#tts = gTTS(text=speech_recognition_system.recommend_sentences_list[-1], lang='en', slow=False)

# Save the generated audio file
#audio_file_path = '/Users/mac/Spiced/Projects/ISP_International_Speech_Project/data/output_audio.mp3'
#tts.save(audio_file_path)

# Play the audio using IPython.display.Audio
audio = Audio(filename='/Users/mac/Spiced/Projects/ISP_International_Speech_Project/audio/output_audio_first.mp3', autoplay=True)
display(audio)

while True:
    speech_recognition_system.recognized_phrase = speech_recognition_system.check_phrase(speech_recognition_system.recommend_sentences_list[-1])

    if speech_recognition_system.recognized_phrase is not None:
        distance = Levenshtein.distance(speech_recognition_system.recommend_sentences_list[-1], speech_recognition_system.recognized_phrase)
        max_length = max(len(speech_recognition_system.recommend_sentences_list[-1]), len(speech_recognition_system.recognized_phrase))
        similarity_ratio = 1 - (distance / max_length)

        similarity_threshold = 0.9

        # Round the similarity ratio to two decimal places
        rounded_similarity_ratio = round(similarity_ratio, 2)

        # Convert to percentage and format the output
        similarity_percentage = f"{rounded_similarity_ratio * 100:.2f}%"

        # ANSI escape code for green color
        green_color = "\033[92m"

        # ANSI escape code to reset color
        reset_color = "\033[0m"

        # Print the formatted output in green
        print(f"{green_color}Similarity Ratio: {similarity_percentage}{reset_color}")
        if similarity_ratio >= similarity_threshold:
            print("The recognized phrase is similar enough to the correct phrase.")

            if len(speech_recognition_system.recommend_sentences_list) < len(texts):
                next_target_sentence = texts[len(speech_recognition_system.recommend_sentences_list)]

                # Create a gTTS object
                tts = gTTS(text=next_target_sentence, lang='en', slow=False)

                # Save the generated audio file
                audio_file_path = '/Users/mac/Spiced/Projects/ISP_International_Speech_Project/audio/output_audio_continue.mp3'
                tts.save(audio_file_path)

                # Play the audio using IPython.display.Audio
                audio = Audio(filename=audio_file_path, autoplay=True)
                display(audio)

                instruction_text = f"Speak the next phrase: {next_target_sentence}"
                speech_recognition_system.recommend_sentences_list.append(next_target_sentence)
            else:
                print("All sentences have been spoken. You can end the loop or perform another action.")
                break
        else:
            print("The recognized phrase is not similar to the correct phrase.")

            speech_recognition_system.print_differences(speech_recognition_system.recommend_sentences_list[-1], speech_recognition_system.recognized_phrase)

            differences = [word[2:] for word in difflib.ndiff(speech_recognition_system.recommend_sentences_list[-1].split(), speech_recognition_system.recognized_phrase.split()) 
                           if word[0] == '-' and len(word[2:]) > 1]
            print("\nIncorrect words:", differences)

            if differences:
                input_word = differences[0]

                rhymes = speech_recognition_system.find_rhymes_in_list(input_word)

                if rhymes:
                    print(f"Rhymes for '{input_word}' in the word list: {', '.join(rhymes)}")
                    rhymes_found = True

                    target_word = rhymes[0]
                    recommend_sentence = speech_recognition_system.find_sentence_with_word(target_word)

                    if recommend_sentence:
                        print(f"The first sentence containing '{target_word}': {recommend_sentence}")

                        # Create a gTTS object
                        tts = gTTS(text=recommend_sentence, lang='en', slow=False)

                        # Save the generated audio file
                        audio_file_path = '/Users/mac/Spiced/Projects/ISP_International_Speech_Project/audio/output_audio.mp3'
                        tts.save(audio_file_path)

                        # Play the audio using IPython.display.Audio
                        audio = Audio(filename=audio_file_path, autoplay=True)
                        display(audio)

                        if recommend_sentence == speech_recognition_system.prev_recommend_sentence:
                            print("The loop will break because the recommend_sentence repeated.")
                            break

                        speech_recognition_system.recommend_sentences_list.append(recommend_sentence)
                        speech_recognition_system.prev_recommend_sentence = recommend_sentence
                    else:
                        print(f"No sentence found containing '{target_word}'.")
                        break
                else:
                    print(f"No rhymes found for '{input_word}' in the word list.")
                    break
            else:
                print("No differences found.")